# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
output_file = "../output_data/cities.csv"

weather_df = pd.read_csv(output_file)

weather_df.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configure gmaps
gmaps.configure(api_key = g_key)

#Store Lat and Lng
locations = weather_df[["Lat", "Lng"]]

#Store Humidity
humidity = weather_df["Humidity"]


In [33]:
# Plot Heatmap
figure = gmaps.figure(center=(10.0, 10.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity =100, point_radius=3)

# Add layer to figure
figure.add_layer(heat_layer)

# Display figure
figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
#Narrow cities with wind speed less than 10 MPH, no clouds, and max temperature between 70 and 90 degrees
ideal_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 90)].dropna()

#Show DF
ideal_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
#Create DF
hotel_df = ideal_city_df.loc[:, ["City", "Country", "Lat", "Lng"]]

#Add empty "Hotel Name" column to DF
hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.rename_axis("City ID")

#Display DF
hotel_df

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
125,arraial do cabo,BR,-22.97,-42.02,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
142,sarai mir,IN,26.02,82.92,
176,nador,MA,35.17,-2.93,
244,pasni,PK,25.26,63.48,
297,mogok,MM,22.92,96.51,


In [48]:
#Define URL
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Define Parameters
params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

#
for index, row in hotel_df.iterrows():
    lattitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lattitude},{longitude}"
    
    print(f"Retrieving Results for City ID {index} | {city}.")
    
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Nearest Hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"]
        
    except (KeyError, IndexError):
            print("No Hotel Found... Skipping.")
        
    
print("........Search has Ended........")
    

Retrieving Results for City ID 70 | ponta do sol.
Nearest Hotel in ponta do sol is Suites de Luxo Paraíso de Minas Escarpas.
Retrieving Results for City ID 88 | cidreira.
Nearest Hotel in cidreira is Hotel Castelo.
Retrieving Results for City ID 125 | arraial do cabo.
Nearest Hotel in arraial do cabo is Orlanova Hotel.
Retrieving Results for City ID 138 | morondava.
Nearest Hotel in morondava is Palissandre Cote Ouest resort & SPA.
Retrieving Results for City ID 139 | vaini.
No Hotel Found... Skipping.
Retrieving Results for City ID 142 | sarai mir.
Nearest Hotel in sarai mir is Bhola T Center.
Retrieving Results for City ID 176 | nador.
Nearest Hotel in nador is Hotel Marchica Lagoon Resort, Nador Morocco.
Retrieving Results for City ID 244 | pasni.
Nearest Hotel in pasni is Juddi Hotel.
Retrieving Results for City ID 297 | mogok.
Nearest Hotel in mogok is Mount Mogok Hotel.
Retrieving Results for City ID 316 | gunjur.
Nearest Hotel in gunjur is Nemasu Ecolodge.
Retrieving Results for

In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))